## Ideas for project:

### Create a "Star Power" metric based on Oscar nominations/wins:

#### Set up a loop to:
* Search each entry in df movies: 
    - (1) save "title_year" (need to change this to int), 
    - (2) save string within column actor_1_name, 
    - (3) search df oscars with conditional statement: if year is < title_year and name is the same, add 1 to counter for number of Oscars (or count the number of entries), 
    - (4) add new column to movies called "actor_1_oscar_nods" and enter the number from the counter
    - (5) repeat for actor_2_name and actor_3_name
* Note that this will count Oscar nominations and wins.  For wins only, set conditional statement to look for "True" under "winner" column

### Another metric of star power would be salary paid to the top actors/actresses:
* Here is a data set of the highest growwing actors of all time, but it's behind a 49 dollar pay wall: https://www.statista.com/statistics/655480/all-time-top-grossing-actors-box-office/
* 

### For release dates, there is a database of 45,000 movies on kaggle that includes full release date:
* https://www.kaggle.com/rounakbanik/the-movies-dataset#ratings.csv
* Try joining with our data set?
* Should we look for dates that are big opening weekend (Memorial Day, Christmas, Thanksgiving, Labor day) and add that as extra weight to release date?
* Has rating and popularity index
* DOes not have top three actor names

# The following code sets up and tests functions to pull Oscar nominations and wins into the main movie df
## Note that in this section, the functions are set up, but then used on small subsets of testing data. 

In [108]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress
import sqlite3


In [26]:
# create connection to database

con = sqlite3.connect("db/movies.db")
sql = f"""
   SELECT * FROM movie_data
   """

# bring in db to pandas dataframe
movies = pd.read_sql(sql, con)
movies.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,release_date,success_bins,production_companies_count,genres_count,popularity,vote_count,vote_average,total_actor_starpower,release_month,holiday_month
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,2009-12-10,extreme success,4.0,4,185.070892,12114.0,7.2,5.334665,12,1
1,Color,Stephen Sommers,106.0,106.0,208.0,855.0,Jason Flemyng,3000.0,11146409.0,Action|Adventure|Horror|Sci-Fi,...,1998-01-30,no success,3.0,4,6.922458,155.0,6.0,10.933174,01,0
2,Color,Terrence Malick,222.0,150.0,0.0,855.0,Michael Greyeyes,23000.0,12712093.0,Biography|Drama|History|Romance,...,2005-12-25,no success,5.0,3,7.694502,336.0,6.4,11.013046,12,1
3,Color,Brian Robbins,76.0,98.0,48.0,722.0,Joel David Moore,21000.0,61112916.0,Comedy|Family|Fantasy,...,2006-03-09,moderate success,3.0,2,4.878907,138.0,4.5,13.698882,03,0
4,Color,Brad Peyton,178.0,94.0,62.0,722.0,Dwayne Johnson,14000.0,103812241.0,Action|Adventure|Comedy|Family|Fantasy|Sci-Fi,...,2012-01-19,average success,3.0,3,9.46307,1050.0,5.8,6.475240,01,0


In [20]:
# # importing movie table
# movies = pd.read_csv("movie_metadata.csv")
# movies.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [ ]:
movies.columns
# Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
#        'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
#        'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
#        'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
#        'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
#        'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
#        'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
#        'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
#       dtype='object')

movies.dtypes


In [11]:
# importing oscar nominations and wins table
oscars = pd.read_csv("oscars.csv")
oscars.head()

,year,category,winner,entity
0,1927,ACTOR,False,Richard Barthelmess
1,1927,ACTOR,True,Emil Jannings
2,1927,ACTRESS,False,Louise Dresser
3,1927,ACTRESS,True,Janet Gaynor
4,1927,ACTRESS,False,Gloria Swanson


In [ ]:
# * Search each entry in df movies: 
#     - (1) save "title_year" (need to change this to int), 
#     - (2) save string within column actor_1_name, 
#     - (3) search df oscars with conditional statement: if year is < title_year and name is the same, add 1 to counter for number of Oscars (or count the number of entries), 
#     - (4) add new column to movies called "actor_1_oscar_nods" and enter the number from the counter
#     - (5) repeat for actor_2_name and actor_3_name
#     - (6) calculate the total noms
#     - (5) repeat for total wins

# * Note that this will count Oscar nominations and wins.  For wins only, set conditional statement to look for "True" under "winner" column
# Note that I can't calculate the oscar wins for the director, as oscar.csv records the directing oscars by movie title, not director


In [ ]:
# use iloc to pull the information on year and actor names
# release_year = int(movies.iloc[0, 23])
# actor1_name = movies.iloc[0, 10] # CCH Pounder
# actor2_name = movies.iloc[0, 6] # Joel David Moore
# actor3_name = movies.iloc[0, 14] # Wes Studi
# actor1_name
# release_year

In [ ]:
# change index to entity
oscars2 = oscars
oscars2 = oscars.set_index("entity")

test_name = oscars[oscars["entity"] == "Meryl Streep"] #new df of all occurances
test_count = oscars2.loc[["Meryl Streep"]].count()
test_nods = len(test_name[(test_name['year'] < release_year)]) #gives int; 15 for Meryl Streep when release year = 2009
test_wins = len(test_name[(test_name['year'] < release_year) & (test_name['winner'] == True)])
test_wins # 2 for Meryl Streep when release_year = 2009
test_nods



In [ ]:
#Note that the string has to be perfect with this syntaxt. So "Robert Downey Jr." will pull out his 2 Oscar noms
# but "Robert Downey Jr" will not:
test_name2 = oscars[oscars["entity"] == "Robert Downey Jr"] #new df of all occurances

# So instead, use str.contains(); regex = False means regex method isn't used when it's not needed, like in the case of a perfect match
test_name2 = oscars[oscars["entity"].str.contains("Robert Downey Jr", regex = False)] #new df of all occurances
test_name2


In [ ]:
# add new column to df and populate with 0
movies2 = movies
# movies["oscar_noms"] = 0
# movies["oscar_wins"] = 0
# movies.head()

# Add number of oscar nominations and wins to proper place in database
# movies.iloc[0, 28] = test_nods
# movies.iloc[0, 29] = test_wins
movies2.head(10)

movies3 = movies2.iloc[0:9]
movies3

In [ ]:
# need to replace NaN in title_year for code to work
movies3.fillna(0, inplace= True)
movies3["title_year"] = movies3["title_year"].astype(int)
# movies3[["title_year"]] = movies3[["title_year"]].fillna(value=0)
movies3

In [ ]:
# import math
# movies3['title_year'] = movies3['title_year'].apply(lambda x : x if math.isnan(x) else int(x)) 

In [ ]:
# Loop through rows of movies3 df to capture info and save date back to 
# test function on a small dataset, movies3 df
for i in range(len(movies4)) : 

    release_year = int(movies4.iloc[i, 23])
#     if movies.iloc[i, 23]:
#         release_year = int(movies.iloc[i, 23])
#     elif ValueError:
#         pass
#     else:
#         pass
#     release_year = movies.iloc[i, 23]
#     if release_year.isdigit():
#       release_year = int(release_year)
#     elif ValueError:
#         pass
#     try:
#         int(release_year)
#     except ValueError:
#         pass
    
    print(release_year)
    actor1_name = movies.iloc[i, 10] # CCH Pounder
    
    actor1_oscars = oscars[oscars["entity"] == actor1_name] #new df of all occurances of actor1's name
    actor1_nods = len(actor1_oscars[(actor1_oscars['year'] < release_year)]) #gives int of # of appearances on list; i.e. total # of noms and wins
    actor1_wins = len(actor1_oscars[(actor1_oscars['year'] < release_year) & (actor1_oscars['winner'] == True)])

    actor2_name = movies.iloc[i, 6] # Joel David Moore
    actor2_oscars = oscars[oscars["entity"] == actor2_name] #new df of all occurances of actor1's name
    actor2_nods = len(actor2_oscars[(actor2_oscars['year'] < release_year)]) #gives int of # of appearances on list; i.e. total # of noms and wins
    actor2_wins = len(actor2_oscars[(actor2_oscars['year'] < release_year) & (actor2_oscars['winner'] == True)])
    
    actor3_name = movies.iloc[i, 14] # Wes Studi
    actor3_oscars = oscars[oscars["entity"] == actor3_name] #new df of all occurances of actor1's name
    actor3_nods = len(actor3_oscars[(actor3_oscars['year'] < release_year)]) #gives int of # of appearances on list; i.e. total # of noms and wins
    actor3_wins = len(actor3_oscars[(actor3_oscars['year'] < release_year) & (actor3_oscars['winner'] == True)])

In [ ]:
movies3

In [ ]:
# def awards(row, actor_num, win=False):
#     global oscars
#     release_year = row['title_year']
#     actor1_name = row[f"actor_{str(actor_num)}_name"] # CCH Pounder
# #     actor2_name = movies3.iloc[0:, 6] # Joel David Moore
# #     actor3_name = movies3.iloc[0:, 14] # Wes Studi
    
#     noms = len(oscars[(oscars['entity'].str.contains(actor1_name, regex = False)) & (oscars['year'] < release_year)]) #gives int; 15 for Meryl Streep when release year = 2009
#     wins = len(oscars[(oscars['entity'].str.contains(actor1_name, regex = False)) & (oscars['year'] < release_year) & (oscars['winner'] == True)])
    
#     return wins if win else noms



In [12]:
def award_noms(row):
    global oscars
    release_year = row['title_year']
    actor1_name = row["actor_1_name"] 
    actor2_name = row["actor_2_name"]  
    actor3_name = row["actor_3_name"]  
    
    noms1 = len(oscars[(oscars['entity'].str.contains(actor1_name, regex = False)) & (oscars['year'] < release_year)]) 
    wins1 = len(oscars[(oscars['entity'].str.contains(actor1_name, regex = False)) & (oscars['year'] < release_year) & (oscars['winner'] == True)])
    
    noms2 = len(oscars[(oscars['entity'].str.contains(actor2_name, regex = False)) & (oscars['year'] < release_year)]) 
    wins2 = len(oscars[(oscars['entity'].str.contains(actor2_name, regex = False)) & (oscars['year'] < release_year) & (oscars['winner'] == True)])
    
    noms3 = len(oscars[(oscars['entity'].str.contains(actor3_name, regex = False)) & (oscars['year'] < release_year)]) 
    wins3 = len(oscars[(oscars['entity'].str.contains(actor3_name, regex = False)) & (oscars['year'] < release_year) & (oscars['winner'] == True)])
    
    tot_noms = noms1 + noms2 + noms3
    tot_wins = wins1 + wins2 + wins3

    return tot_noms



In [ ]:
movies3["tot_noms"] = movies3.apply(award_noms, axis=1)

In [ ]:
movies3

In [13]:
def award_wins(row):
    global oscars
    release_year = row['title_year']
    actor1_name = row["actor_1_name"] 
    actor2_name = row["actor_2_name"]  
    actor3_name = row["actor_3_name"]  
    
    noms1 = len(oscars[(oscars['entity'].str.contains(actor1_name, regex = False)) & (oscars['year'] < release_year)]) 
    wins1 = len(oscars[(oscars['entity'].str.contains(actor1_name, regex = False)) & (oscars['year'] < release_year) & (oscars['winner'] == True)])
    
    noms2 = len(oscars[(oscars['entity'].str.contains(actor2_name, regex = False)) & (oscars['year'] < release_year)]) 
    wins2 = len(oscars[(oscars['entity'].str.contains(actor2_name, regex = False)) & (oscars['year'] < release_year) & (oscars['winner'] == True)])
    
    noms3 = len(oscars[(oscars['entity'].str.contains(actor3_name, regex = False)) & (oscars['year'] < release_year)]) 
    wins3 = len(oscars[(oscars['entity'].str.contains(actor3_name, regex = False)) & (oscars['year'] < release_year) & (oscars['winner'] == True)])
    
    tot_noms = noms1 + noms2 + noms3
    tot_wins = wins1 + wins2 + wins3

    return tot_wins


In [ ]:
movies3["tot_wins"] = movies3.apply(award_wins, axis=1)

In [ ]:
movies3

# Re-run functions above for entire movies dataset:

In [27]:
# edit original df to change NaN to zero and change float to int
movies2 = movies.copy(deep=True)
movies2.fillna(0, inplace= True)
movies2["title_year"] = movies2["title_year"].astype(int)
movies2.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,release_date,success_bins,production_companies_count,genres_count,popularity,vote_count,vote_average,total_actor_starpower,release_month,holiday_month
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,2009-12-10,extreme success,4.0,4,185.070892,12114.0,7.2,5.334665,12,1
1,Color,Stephen Sommers,106.0,106.0,208.0,855.0,Jason Flemyng,3000.0,11146409.0,Action|Adventure|Horror|Sci-Fi,...,1998-01-30,no success,3.0,4,6.922458,155.0,6.0,10.933174,01,0
2,Color,Terrence Malick,222.0,150.0,0.0,855.0,Michael Greyeyes,23000.0,12712093.0,Biography|Drama|History|Romance,...,2005-12-25,no success,5.0,3,7.694502,336.0,6.4,11.013046,12,1
3,Color,Brian Robbins,76.0,98.0,48.0,722.0,Joel David Moore,21000.0,61112916.0,Comedy|Family|Fantasy,...,2006-03-09,moderate success,3.0,2,4.878907,138.0,4.5,13.698882,03,0
4,Color,Brad Peyton,178.0,94.0,62.0,722.0,Dwayne Johnson,14000.0,103812241.0,Action|Adventure|Comedy|Family|Fantasy|Sci-Fi,...,2012-01-19,average success,3.0,3,9.46307,1050.0,5.8,6.475240,01,0


In [ ]:
# functions are the same, just need to call the movies2 df now to get data for all 5000 movies

In [28]:
movies2["tot_noms"] = movies2.apply(award_noms, axis=1)
movies2["tot_wins"] = movies2.apply(award_wins, axis=1)
movies2.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,production_companies_count,genres_count,popularity,vote_count,vote_average,total_actor_starpower,release_month,holiday_month,tot_noms,tot_wins
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,4.0,4,185.070892,12114.0,7.2,5.334665,12,1,0,0
1,Color,Stephen Sommers,106.0,106.0,208.0,855.0,Jason Flemyng,3000.0,11146409.0,Action|Adventure|Horror|Sci-Fi,...,3.0,4,6.922458,155.0,6.0,10.933174,01,0,0,0
2,Color,Terrence Malick,222.0,150.0,0.0,855.0,Michael Greyeyes,23000.0,12712093.0,Biography|Drama|History|Romance,...,5.0,3,7.694502,336.0,6.4,11.013046,12,1,0,0
3,Color,Brian Robbins,76.0,98.0,48.0,722.0,Joel David Moore,21000.0,61112916.0,Comedy|Family|Fantasy,...,3.0,2,4.878907,138.0,4.5,13.698882,03,0,1,0
4,Color,Brad Peyton,178.0,94.0,62.0,722.0,Dwayne Johnson,14000.0,103812241.0,Action|Adventure|Comedy|Family|Fantasy|Sci-Fi,...,3.0,3,9.46307,1050.0,5.8,6.475240,01,0,0,0


In [29]:
movies.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,release_date,success_bins,production_companies_count,genres_count,popularity,vote_count,vote_average,total_actor_starpower,release_month,holiday_month
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,2009-12-10,extreme success,4.0,4,185.070892,12114.0,7.2,5.334665,12,1
1,Color,Stephen Sommers,106.0,106.0,208.0,855.0,Jason Flemyng,3000.0,11146409.0,Action|Adventure|Horror|Sci-Fi,...,1998-01-30,no success,3.0,4,6.922458,155.0,6.0,10.933174,01,0
2,Color,Terrence Malick,222.0,150.0,0.0,855.0,Michael Greyeyes,23000.0,12712093.0,Biography|Drama|History|Romance,...,2005-12-25,no success,5.0,3,7.694502,336.0,6.4,11.013046,12,1
3,Color,Brian Robbins,76.0,98.0,48.0,722.0,Joel David Moore,21000.0,61112916.0,Comedy|Family|Fantasy,...,2006-03-09,moderate success,3.0,2,4.878907,138.0,4.5,13.698882,03,0
4,Color,Brad Peyton,178.0,94.0,62.0,722.0,Dwayne Johnson,14000.0,103812241.0,Action|Adventure|Comedy|Family|Fantasy|Sci-Fi,...,2012-01-19,average success,3.0,3,9.46307,1050.0,5.8,6.475240,01,0


In [30]:
movies["tot_noms"] = movies2["tot_noms"]
movies.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,success_bins,production_companies_count,genres_count,popularity,vote_count,vote_average,total_actor_starpower,release_month,holiday_month,tot_noms
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,extreme success,4.0,4,185.070892,12114.0,7.2,5.334665,12,1,0
1,Color,Stephen Sommers,106.0,106.0,208.0,855.0,Jason Flemyng,3000.0,11146409.0,Action|Adventure|Horror|Sci-Fi,...,no success,3.0,4,6.922458,155.0,6.0,10.933174,01,0,0
2,Color,Terrence Malick,222.0,150.0,0.0,855.0,Michael Greyeyes,23000.0,12712093.0,Biography|Drama|History|Romance,...,no success,5.0,3,7.694502,336.0,6.4,11.013046,12,1,0
3,Color,Brian Robbins,76.0,98.0,48.0,722.0,Joel David Moore,21000.0,61112916.0,Comedy|Family|Fantasy,...,moderate success,3.0,2,4.878907,138.0,4.5,13.698882,03,0,1
4,Color,Brad Peyton,178.0,94.0,62.0,722.0,Dwayne Johnson,14000.0,103812241.0,Action|Adventure|Comedy|Family|Fantasy|Sci-Fi,...,average success,3.0,3,9.46307,1050.0,5.8,6.475240,01,0,0


In [31]:
movies["tot_wins"] = movies2["tot_wins"]

movies.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,production_companies_count,genres_count,popularity,vote_count,vote_average,total_actor_starpower,release_month,holiday_month,tot_noms,tot_wins
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,4.0,4,185.070892,12114.0,7.2,5.334665,12,1,0,0
1,Color,Stephen Sommers,106.0,106.0,208.0,855.0,Jason Flemyng,3000.0,11146409.0,Action|Adventure|Horror|Sci-Fi,...,3.0,4,6.922458,155.0,6.0,10.933174,01,0,0,0
2,Color,Terrence Malick,222.0,150.0,0.0,855.0,Michael Greyeyes,23000.0,12712093.0,Biography|Drama|History|Romance,...,5.0,3,7.694502,336.0,6.4,11.013046,12,1,0,0
3,Color,Brian Robbins,76.0,98.0,48.0,722.0,Joel David Moore,21000.0,61112916.0,Comedy|Family|Fantasy,...,3.0,2,4.878907,138.0,4.5,13.698882,03,0,1,0
4,Color,Brad Peyton,178.0,94.0,62.0,722.0,Dwayne Johnson,14000.0,103812241.0,Action|Adventure|Comedy|Family|Fantasy|Sci-Fi,...,3.0,3,9.46307,1050.0,5.8,6.475240,01,0,0,0


In [32]:
# store dataframe in a sqlite db
con = sqlite3.connect("db/movies.db")
movies.to_sql("movie_data", con, if_exists="replace", index=False)
# commit the changes and close the connection
con.commit()
con.close()

In [33]:
# store dataframe in a sqlite db
con = sqlite3.connect("db/movies.db")
oscars.to_sql("oscars", con, if_exists="replace", index=False)
# commit the changes and close the connection
con.commit()
con.close()

In [34]:
# create connection to database

con = sqlite3.connect("db/movies.db")
sql = f"""
   SELECT * FROM movie_data
   """

# bring in db to pandas dataframe
movieeees = pd.read_sql(sql, con)
movieeees.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,production_companies_count,genres_count,popularity,vote_count,vote_average,total_actor_starpower,release_month,holiday_month,tot_noms,tot_wins
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,4.0,4,185.070892,12114.0,7.2,5.334665,12,1,0,0
1,Color,Stephen Sommers,106.0,106.0,208.0,855.0,Jason Flemyng,3000.0,11146409.0,Action|Adventure|Horror|Sci-Fi,...,3.0,4,6.922458,155.0,6.0,10.933174,01,0,0,0
2,Color,Terrence Malick,222.0,150.0,0.0,855.0,Michael Greyeyes,23000.0,12712093.0,Biography|Drama|History|Romance,...,5.0,3,7.694502,336.0,6.4,11.013046,12,1,0,0
3,Color,Brian Robbins,76.0,98.0,48.0,722.0,Joel David Moore,21000.0,61112916.0,Comedy|Family|Fantasy,...,3.0,2,4.878907,138.0,4.5,13.698882,03,0,1,0
4,Color,Brad Peyton,178.0,94.0,62.0,722.0,Dwayne Johnson,14000.0,103812241.0,Action|Adventure|Comedy|Family|Fantasy|Sci-Fi,...,3.0,3,9.46307,1050.0,5.8,6.475240,01,0,0,0


In [35]:
# create connection to database

con = sqlite3.connect("db/movies.db")
sql = f"""
   SELECT * FROM oscars
   """

# bring in db to pandas dataframe
oscarzz = pd.read_sql(sql, con)
oscarzz.head()

,year,category,winner,entity
0,1927,ACTOR,0,Richard Barthelmess
1,1927,ACTOR,1,Emil Jannings
2,1927,ACTRESS,0,Louise Dresser
3,1927,ACTRESS,1,Janet Gaynor
4,1927,ACTRESS,0,Gloria Swanson


# Testing models:
## Prepping the data to be tested by the QDA model

In [109]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.metrics import mean_squared_error, r2_score, roc_curve, roc_auc_score, f1_score, precision_recall_curve, average_precision_score, auc


In [153]:
# create connection to database

con = sqlite3.connect("db/movies.db")
sql = f"""
   SELECT * FROM movie_data
   """

# bring in db to pandas dataframe
movieDf = pd.read_sql(sql, con)
movieDf.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,production_companies_count,genres_count,popularity,vote_count,vote_average,total_actor_starpower,release_month,holiday_month,tot_noms,tot_wins
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,4.0,4,185.070892,12114.0,7.2,5.334665,12,1,0,0
1,Color,Stephen Sommers,106.0,106.0,208.0,855.0,Jason Flemyng,3000.0,11146409.0,Action|Adventure|Horror|Sci-Fi,...,3.0,4,6.922458,155.0,6.0,10.933174,01,0,0,0
2,Color,Terrence Malick,222.0,150.0,0.0,855.0,Michael Greyeyes,23000.0,12712093.0,Biography|Drama|History|Romance,...,5.0,3,7.694502,336.0,6.4,11.013046,12,1,0,0
3,Color,Brian Robbins,76.0,98.0,48.0,722.0,Joel David Moore,21000.0,61112916.0,Comedy|Family|Fantasy,...,3.0,2,4.878907,138.0,4.5,13.698882,03,0,1,0
4,Color,Brad Peyton,178.0,94.0,62.0,722.0,Dwayne Johnson,14000.0,103812241.0,Action|Adventure|Comedy|Family|Fantasy|Sci-Fi,...,3.0,3,9.46307,1050.0,5.8,6.475240,01,0,0,0


In [152]:
# review columns
for col in movieDf.columns.sort_values().values:
    print(col)

actor_1_facebook_likes
actor_1_name
actor_1_score
actor_2_facebook_likes
actor_2_name
actor_2_score
actor_3_facebook_likes
actor_3_name
actor_3_score
aspect_ratio
budget
cast_total_facebook_likes
color
content_rating
country
director_facebook_likes
director_name
director_score
duration
facenumber_in_poster
genres
genres_count
gross
gross_margin
holiday_month
imdb_num_code
imdb_score
language
movie_facebook_likes
movie_imdb_link
movie_title
num_critic_for_reviews
num_user_for_reviews
num_voted_users
plot_keywords
popularity
production_companies_count
rating_numeric
release_date
release_month
revenue
success_bins
successful
title_year
tot_noms
tot_wins
total_actor_starpower
vote_average
vote_count


In [154]:
# create table for model purposes
# Note that I haven't included any predictors that won't be known before a movie release
movieDf = movieDf[["duration","facenumber_in_poster","rating_numeric","gross_margin", 
                   "successful","director_score","production_companies_count","total_actor_starpower","release_month",
                   "holiday_month","tot_wins","tot_noms", "total_actor_starpower", "vote_average", "vote_count"]]
movieDf = movieDf.dropna()
movieDf.head(10)


,duration,facenumber_in_poster,rating_numeric,gross_margin,successful,director_score,production_companies_count,total_actor_starpower,release_month,holiday_month,tot_wins,tot_noms,total_actor_starpower,vote_average,vote_count
0,178.0,0.0,3,0.688365,1,3.091853,4.0,5.334665,12,1,0,0,5.334665,7.2,12114.0
1,106.0,0.0,4,-3.037175,0,1.766773,3.0,10.933174,01,0,0,0,10.933174,6.0,155.0
2,150.0,0.0,3,-1.359958,0,1.325080,5.0,11.013046,12,1,0,0,11.013046,6.4,336.0
3,98.0,0.0,2,0.181842,1,2.650160,3.0,13.698882,03,0,0,1,13.698882,4.5,138.0
4,94.0,3.0,2,0.239011,1,1.325080,3.0,6.475240,01,0,0,0,6.475240,5.8,1050.0
5,146.0,4.0,4,-0.048999,0,0.441693,4.0,3.409212,05,0,1,2,3.409212,5.5,434.0
6,90.0,0.0,3,-0.325528,0,2.650160,1.0,2.816560,09,0,0,0,2.816560,4.9,134.0
7,121.0,0.0,4,-0.241362,0,2.208466,1.0,6.499734,11,1,0,0,6.499734,5.5,488.0
8,101.0,1.0,4,-2.179885,0,0.883387,8.0,3.409212,08,0,0,0,3.409212,3.2,106.0
9,110.0,1.0,4,0.357153,1,0.883387,4.0,3.409212,08,0,0,0,3.409212,6.5,1950.0


In [155]:
movieDf["successful"].value_counts()

1    1950
0    1687
Name: successful, dtype: int64

In [156]:
# Assign X (data) and y (target)
# Change X if you want to test other parameters
X = movieDf[["rating_numeric","director_score","production_companies_count","total_actor_starpower","release_month",
             "holiday_month","tot_wins","tot_noms", "total_actor_starpower", "vote_average", "vote_count"]]
y = movieDf["successful"].values.reshape(-1, 1)
print(X.shape, y.shape)


(3637, 13) (3637, 1)


### Summary of test scores with different combos of features in QDA:
X = modelDf[["rating_numeric","director_score","production_companies_count","total_actor_starpower","release_month",
             "holiday_month","tot_wins","tot_noms", "total_actor_starpower", "vote_average", "vote_count"]]
testing_score: 0.47802197802197804, mse: 0.521978021978022, r2: -1.0986193293885602

**
X = modelDf[["director_score","production_companies_count","total_actor_starpower",
             "tot_noms", "total_actor_starpower", "vote_average"]]
testing_score: 0.5096153846153846, mse: 0.49038461538461536, r2: -0.9715976331360949
BETTER SCORE WITHOUT RATING, RELEASE MONTH, HOLIDAY MONTH, TOT WINS, VOTE COUNT; ARE ALL OF THESE HARMFUL IN THE FIRST COMBO OF ALL FEATURES

X = modelDf[["director_score","production_companies_count","total_actor_starpower",
             "tot_noms", "total_actor_starpower"]]
testing_score: 0.46703296703296704, mse: 0.532967032967033, r2: -1.1428007889546352
VOTE_AVERAGE WAS HELPFUL

X = modelDf[["director_score","production_companies_count","total_actor_starpower",
             "tot_noms", "total_actor_starpower", "vote_average", "vote_count"]]
testing_score: 0.4807692307692308, mse: 0.5192307692307693, r2: -1.0875739644970417
VOTE COUNT WAS LESS HELPFUL WHEN ADDED TO VOTE AVG

X = modelDf[["director_score","production_companies_count","total_actor_starpower",
             "tot_noms", "total_actor_starpower", "vote_count"]]
testing_score: 0.46565934065934067, mse: 0.5343406593406593, r2: -1.1483234714003947
VOTE COUNT IS WORSE THAN VOTE AVERAGE

***
X = modelDf[["director_score","production_companies_count","total_actor_starpower",
             "tot_noms",  "vote_average"]]
testing_score: 0.5686813186813187, mse: 0.43131868131868134, r2: -0.7341222879684419
CUTTING TOTAL ACTOR STAR POWER WAS HELPFUL

X = modelDf[["director_score","production_companies_count","total_actor_starpower",
              "vote_average"]]
testing_score: 0.47115384615384615, mse: 0.5288461538461539, r2: -1.1262327416173572
TOT NOMS WAS HELPFUL

****
X = modelDf[["director_score","production_companies_count",
             "tot_noms",  "vote_average"]]
testing_score: 0.603021978021978, mse: 0.39697802197802196, r2: -0.5960552268244577
CUTTING "total_actor_starpower" WAS HELPFUL

X = modelDf[["director_score",
             "tot_noms",  "vote_average"]]
testing_score: 0.6016483516483516, mse: 0.3983516483516483, r2: -0.6015779092702171
CUTTING "production_companies_count", DIDN'T AFFECT THINGS MUCH

X = modelDf[["production_companies_count",
             "tot_noms",  "vote_average"]]
testing_score: 0.6002747252747253, mse: 0.39972527472527475, r2: -0.6071005917159764
CUTTING "director_score" LOWERED THE SCORE SLIGHTLY

*****
X = modelDf[["rating_numeric", "director_score","production_companies_count",
             "tot_noms",  "vote_average"]]
testing_score: 0.6043956043956044, mse: 0.3956043956043956, r2: -0.5905325443786984
ADDING rating_numeric INCREASED THE SCORE SLIGHTLY

X = modelDf[["release_month", "director_score","production_companies_count",
             "tot_noms",  "vote_average"]]
testing_score: 0.6016483516483516, mse: 0.3983516483516483, r2: -0.6015779092702171
release_month LOWERED THE SCORE

X = modelDf[["holiday_month", "director_score","production_companies_count",
             "tot_noms",  "vote_average"]]
testing_score: 0.6057692307692307, mse: 0.3942307692307692, r2: -0.5850098619329389
ADDING holiday_month INCREASED SCORE SLIGHTLY

X = modelDf[["tot_wins", "director_score","production_companies_count",
             "tot_noms",  "vote_average"]]
testing_score: 0.5961538461538461, mse: 0.40384615384615385, r2: -0.6236686390532546
tot_wins LOWERED SCORES

****** BEST SCORE DO FAR:
X = modelDf[["vote_count", "director_score","production_companies_count",
             "tot_noms",  "vote_average"]]
testing_score: 0.6126373626373627, mse: 0.3873626373626374, r2: -0.5573964497041421
ADDING vote_count GAVE NEW HIGH SCORE, BUT NEED TO CHECK IF VOTE AVG AND VOTE COUNT ARE CONNECTED 

In [141]:
# split the training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1158, train_size=0.80, stratify=y)

# Create a StandardScater model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  Ignored


In [142]:
pd.DataFrame(y_test)[0].value_counts()

1    390
0    338
Name: 0, dtype: int64

In [143]:
# transform the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


## QDA (Quadratic Discriminant Analysis)

In [105]:
#build logistic model 
model = QuadraticDiscriminantAnalysis()

#fit on training data
model.fit(X_train_scaled, y_train)


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
               store_covariance=False, store_covariances=None, tol=0.0001)

In [106]:
# Use our model to predict a value
predicted = model.predict(X_test_scaled)

# Score the prediction with mse and r2
mse = mean_squared_error(y_test, predicted)
r2 = r2_score(y_test, predicted)
testing_score = model.score(X_test_scaled,y_test)

print(f"testing_score: {testing_score}, mse: {mse}, r2: {r2}")

testing_score: 0.6126373626373627, mse: 0.3873626373626374, r2: -0.5573964497041421


In [107]:
# QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
#                               store_covariance=False, tol=0.0001)
# print(clf.predict([[-0.8, -1]]))

# Trying the PCA analysis with QDA model

In [114]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn import decomposition
from sklearn import datasets
from sklearn.svm import SVC

In [133]:
pca = decomposition.PCA(n_components = 2)

In [144]:
pca.fit(X_train_scaled, y_train)

PCA(copy=True, iterated_power='auto', n_components=6, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [145]:
X_train_pca = pca.transform(X_train_scaled)

In [146]:
len(X_train_pca[0])

6

In [147]:
# define model; CHANGE FOR YOUR MODEL
# for SVC:
# modelSVC = SVC(C=5, gamma=0.1, kernel='linear')

model = QuadraticDiscriminantAnalysis()

# fit model with PCA training data
model.fit(X_train_pca, y_train)

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
               store_covariance=False, store_covariances=None, tol=0.0001)

In [148]:
X_test_pca = pca.transform(X_test_scaled)
X_test_pca

array([[-1.11749042, -0.59883531,  0.48446856, -0.39149464, -0.77514173,
         1.20677506],
       [ 0.17297186,  2.11991443, -0.60563626, -0.9663372 , -1.15125246,
         0.52345433],
       [ 7.19545337,  1.64361411,  5.97623882,  8.00228485, -0.39043428,
        -1.1876921 ],
       ...,
       [ 1.59094877,  1.90423495, -0.58642724, -1.58204162,  0.37464378,
         0.82722126],
       [ 7.99712765,  3.95802517, 12.0810899 , 15.32142302, -1.34059089,
        -2.83106286],
       [-1.56796968,  0.87766099, -1.34277442,  1.25415834, -1.15396034,
         1.68708421]])

In [149]:
model.score(X_test_pca, y_test)

0.635989010989011

In [150]:
# Function to test PCA with a range of component values

for n in range(2,10):
    
    pca = decomposition.PCA(n_components = n)
    pca.fit(X_train_scaled, y_train)
    X_train_pca = pca.transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)
    
    model.fit(X_train_pca, y_train)
    print("n="+str(n),model.score(X_test_pca, y_test),"")

n=2 0.5439560439560439 
n=3 0.5796703296703297 
n=4 0.5824175824175825 
n=5 0.5947802197802198 
n=6 0.635989010989011 
n=7 0.6442307692307693 
n=8 0.6318681318681318 
n=9 0.6304945054945055 
n=10 0.6222527472527473 
n=11 0.6263736263736264 


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please chan

### Notes on PCA analysis with QDA
When X is defined as:
X = modelDf[["holiday_month", "director_score","production_companies_count",
             "tot_noms",  "vote_average"]]
and PCA range is 2-6, the test scores are:
n=2 0.5879120879120879 
n=3 0.6126373626373627 
n=4 0.592032967032967 
n=5 0.6057692307692307 

So, some combo of 3 features is the best.  

If I go back and define X with all the features:
X = modelDf[["rating_numeric","director_score","production_companies_count","total_actor_starpower","release_month",
             "holiday_month","tot_wins","tot_noms", "total_actor_starpower", "vote_average", "vote_count"]]
             
and PCA range is 2-12, the test scores are:
n=2 0.5439560439560439 
n=3 0.5796703296703297 
n=4 0.5824175824175825 
n=5 0.5947802197802198 
n=6 0.635989010989011  **
n=7 0.6442307692307693 ***
n=8 0.6318681318681318 **
n=9 0.6304945054945055 **
n=10 0.6222527472527473 
n=11 0.6263736263736264 

In [ ]:
# how to determine which 6-9 features give these test scores?
# see https://stackoverflow.com/questions/22984335/recovering-features-names-of-explained-variance-ratio-in-pca-with-sklearn

